In [25]:
from dash_website.utils.aws_loader import load_csv, upload_file
import numpy as np


SAMPLE_DEFINITION_NAMING = {
    "instances": "all_samples_per_participant",
    "eids": "average_per_participant",
}
COLUMNS_TO_TAKE = {
    "organ": "dimension",
    "view": "subdimension",
    "transformation": "sub_subdimension",
    "architecture": "algorithm",
    "N_all": "sample_size",
    "R-Squared_str_all": "r2_and_std",
    "C-Index_str_all": "c_index_and_std",
    "RMSE_str_all": "rmse_and_std",
    "C-Index-difference_str_all": "c_index_difference_and_std"
}
DICT_TO_CHANGE_DIMENSIONS = {"ImmuneSystem": "BloodCells"}


if __name__ == "__main__":
    for sample_definition in ["instances", "eids"]:
        scores = load_csv(
            f"page2_predictions/Performances/PERFORMANCES_withEnsembles_withCI_alphabetical_{sample_definition}_Age_test.csv"
        )[COLUMNS_TO_TAKE].rename(columns=COLUMNS_TO_TAKE)

        for metric in ["r2", "rmse", "c_index", "c_index_difference"]:
            scores[metric] = scores[f"{metric}_and_std"].str.split("+", expand=True)[0].astype(np.float32)
            scores[f"{metric}_std"] = scores[f"{metric}_and_std"].str.split("+", expand=True)[1].str.split("-", expand=True)[1].astype(np.float32)
        
            scores.drop(columns=f"{metric}_and_std", inplace=True)

        scores.loc[
            (scores["dimension"] == "Musculoskeletal") & (scores["sub_subdimension"] == "MRI"), "sub_subdimension"
        ] = "DXA"
        scores.replace(DICT_TO_CHANGE_DIMENSIONS)# .to_feather(
        #     f"all_data/age_prediction_performances/scores_{SAMPLE_DEFINITION_NAMING[sample_definition]}.feather"
        # )
        # upload_file(f"all_data/age_prediction_performances/scores_{SAMPLE_DEFINITION_NAMING[sample_definition]}.feather", f"age_prediction_performances/scores_{SAMPLE_DEFINITION_NAMING[sample_definition]}.feather")

In [23]:
from dash_website.age_prediction_performances import CUSTOM_DIMENSIONS
scores.set_index(
        ["dimension", "subdimension", "sub_subdimension"], drop=False
    , inplace=True)
scores.drop(index=scores.index[~scores.index.isin(CUSTOM_DIMENSIONS)], inplace=True)
scores.replace("1DCNN", "*", inplace=True)  # since it is the only one that is different
scores.drop(index=scores.index[scores["algorithm"] != "*"], inplace=True)


In [27]:
scores.set_index(
        ["dimension", "subdimension", "sub_subdimension"], drop=False
    , inplace=True)

In [29]:
scores.loc[("Heart", "ECG")]

,dimension,subdimension,sub_subdimension,algorithm,sample_size,r2,r2_std,rmse,rmse_std,c_index,c_index_std,c_index_difference,c_index_difference_std
sub_subdimension,,,,,,,,,,,,,
*,Heart,ECG,*,*,21793,0.377,0.018,5.905,0.089,0.715,0.134,-0.002,0.040
Scalars,Heart,ECG,Scalars,*,21800,0.111,0.016,7.053,0.065,0.743,0.120,0.026,0.050
Scalars,Heart,ECG,Scalars,ElasticNet,21800,0.087,0.016,7.145,0.051,0.749,0.119,0.032,0.049
Scalars,Heart,ECG,Scalars,LightGBM,21800,0.108,0.020,7.065,0.080,0.740,0.121,0.023,0.059
Scalars,Heart,ECG,Scalars,NeuralNetwork,21800,0.094,0.014,7.118,0.052,0.752,0.122,0.035,0.051
TimeSeries,Heart,ECG,TimeSeries,*,42357,0.355,0.015,6.085,0.067,0.717,0.083,0.021,0.045
TimeSeries,Heart,ECG,TimeSeries,1DCNN,42357,0.355,0.015,6.085,0.067,0.717,0.083,0.021,0.045
